# Exploring PySpark

### Initiating a Spark Session

In [139]:
# import SparkSession
from pyspark.sql import SparkSession

# Create SparkApplication
spark = SparkSession.builder.appName('test').getOrCreate()

In [140]:
# Spark details
spark

### Loading Data

In [207]:
# Read the CSV file with header
df = spark.read.option('header', 'true').csv('heart.csv')

# Specify the schema
schema = 'Age INTEGER, Sex STRING, ChestPainType STRING'
df = spark.read.csv('heart.csv', schema=schema, header=True)

# Infer the schema
# df = spark.read.csv('heart.csv', inferSchema=True, header=True)

# Replace null values with 'NA'
df = spark.read.csv('heart.csv', nullValue='NA')

# Save the DataFrame as a CSV file (overwrite mode)
df.write.format("csv").mode("overwrite").save("heart_save.csv")


### Data Exploring

In [176]:
df = spark.read.option('header', 'true').csv('heart.csv')
df = df.select("Age", "Sex", "ChestPainType")
df.show(3)

+---+---+-------------+
|Age|Sex|ChestPainType|
+---+---+-------------+
| 40|  M|          ATA|
| 49|  F|          NAP|
| 37|  M|          ATA|
+---+---+-------------+
only showing top 3 rows



In [177]:
# count number of rows
df.count()

918

In [178]:
# be avare with a large amount of data !
# df.cache()
# df.collect()

In [179]:
# convert PySpark DataFrame to Pandas DataFrame
pd_df = df.toPandas()
# convert it back
spark_df = spark.createDataFrame(pd_df)

In [180]:
df.head(3)

[Row(Age='40', Sex='M', ChestPainType='ATA'),
 Row(Age='49', Sex='F', ChestPainType='NAP'),
 Row(Age='37', Sex='M', ChestPainType='ATA')]

In [181]:
# type os columns
df.printSchema()

root
 |-- Age: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- ChestPainType: string (nullable = true)



In [182]:
# column dtypes
df.dtypes

[('Age', 'string'), ('Sex', 'string'), ('ChestPainType', 'string')]

In [183]:
# cast a column from one type to other
from pyspark.sql.types import FloatType
df = df.withColumn("Age",df.Age.cast(FloatType()))
df = df.withColumn("RestingBP",df.Age.cast(FloatType()))

In [184]:
# compute summery statistics
df.select(['Age','RestingBP']).describe().show()

+-------+------------------+------------------+
|summary|               Age|         RestingBP|
+-------+------------------+------------------+
|  count|               918|               918|
|   mean|53.510893246187365|53.510893246187365|
| stddev|  9.43261650673202|  9.43261650673202|
|    min|              28.0|              28.0|
|    max|              77.0|              77.0|
+-------+------------------+------------------+



In [185]:
# add a new column or replace existing one
AgeFixed = df['Age'] + 1
df = df.withColumn('AgeFixed', AgeFixed)

In [186]:
df.select(['AgeFixed','Age']).describe().show()

+-------+------------------+------------------+
|summary|          AgeFixed|               Age|
+-------+------------------+------------------+
|  count|               918|               918|
|   mean|54.510893246187365|53.510893246187365|
| stddev|  9.43261650673202|  9.43261650673202|
|    min|              29.0|              28.0|
|    max|              78.0|              77.0|
+-------+------------------+------------------+



In [187]:
# remove columns
df.drop('AgeFixed').show(1)

+----+---+-------------+---------+
| Age|Sex|ChestPainType|RestingBP|
+----+---+-------------+---------+
|40.0|  M|          ATA|     40.0|
+----+---+-------------+---------+
only showing top 1 row



In [188]:
# rename a column
df.withColumnRenamed('Age','age').select('age').show(1)
# renaming mulitple columns with a loop
name_pairs = [('Age','age'),('Sex','sex')]
for old_name, new_name in name_pairs:
    df = df.withColumnRenamed(old_name,new_name)

+----+
| age|
+----+
|40.0|
+----+
only showing top 1 row



In [189]:
df.select(['age','sex']).show(1)

+----+---+
| age|sex|
+----+---+
|40.0|  M|
+----+---+
only showing top 1 row



### Cleaning Data

In [190]:
# drop NA
df = df.na.drop()
df.count()
# drop all NA
df = df.na.drop(how='all')
# drop all rows where more at least 2 values are NOT NA
df = df.na.drop(thresh=2)
# drop all rows where any value at specific columns are NAs.
df = df.na.drop(how='any', subset=['age','sex'])

In [191]:
# imputation
df = df.na.fill(value='?',subset=['sex'])
# replace NAs with mean of column
from pyspark.ml.feature import Imputer

imptr = Imputer(inputCols=['age','RestingBP'],
                outputCols=['age','RestingBP']).setStrategy('mean')

df = imptr.fit(df).transform(df)

In [192]:
# filtering
df.filter('age > 18')
df.where('age > 18') # or df.where(df['age'] > 18)

# filtering with condition
df.where((df['age'] > 18) | (df['ChestPainType'] == 'ATA'))
df.filter(~(df['ChestPainType'] == 'ATA'))

DataFrame[age: float, sex: string, ChestPainType: string, RestingBP: float, AgeFixed: float]

In [193]:
df.filter('age > 18').show()

+----+---+-------------+---------+--------+
| age|sex|ChestPainType|RestingBP|AgeFixed|
+----+---+-------------+---------+--------+
|40.0|  M|          ATA|     40.0|    41.0|
|49.0|  F|          NAP|     49.0|    50.0|
|37.0|  M|          ATA|     37.0|    38.0|
|48.0|  F|          ASY|     48.0|    49.0|
|54.0|  M|          NAP|     54.0|    55.0|
|39.0|  M|          NAP|     39.0|    40.0|
|45.0|  F|          ATA|     45.0|    46.0|
|54.0|  M|          ATA|     54.0|    55.0|
|37.0|  M|          ASY|     37.0|    38.0|
|48.0|  F|          ATA|     48.0|    49.0|
|37.0|  F|          NAP|     37.0|    38.0|
|58.0|  M|          ATA|     58.0|    59.0|
|39.0|  M|          ATA|     39.0|    40.0|
|49.0|  M|          ASY|     49.0|    50.0|
|42.0|  F|          NAP|     42.0|    43.0|
|54.0|  F|          ATA|     54.0|    55.0|
|38.0|  M|          ASY|     38.0|    39.0|
|43.0|  F|          ATA|     43.0|    44.0|
|60.0|  M|          ASY|     60.0|    61.0|
|36.0|  M|          ATA|     36.

In [194]:
# evaluate a string expression into command
from pyspark.sql.functions import expr
exp = 'age + 0.2 * AgeFixed'
df.withColumn('new_col', expr(exp)).select('new_col').show(3)

+-------+
|new_col|
+-------+
|   48.2|
|   59.0|
|   44.6|
+-------+
only showing top 3 rows



In [195]:
df.show()

+----+---+-------------+---------+--------+
| age|sex|ChestPainType|RestingBP|AgeFixed|
+----+---+-------------+---------+--------+
|40.0|  M|          ATA|     40.0|    41.0|
|49.0|  F|          NAP|     49.0|    50.0|
|37.0|  M|          ATA|     37.0|    38.0|
|48.0|  F|          ASY|     48.0|    49.0|
|54.0|  M|          NAP|     54.0|    55.0|
|39.0|  M|          NAP|     39.0|    40.0|
|45.0|  F|          ATA|     45.0|    46.0|
|54.0|  M|          ATA|     54.0|    55.0|
|37.0|  M|          ASY|     37.0|    38.0|
|48.0|  F|          ATA|     48.0|    49.0|
|37.0|  F|          NAP|     37.0|    38.0|
|58.0|  M|          ATA|     58.0|    59.0|
|39.0|  M|          ATA|     39.0|    40.0|
|49.0|  M|          ASY|     49.0|    50.0|
|42.0|  F|          NAP|     42.0|    43.0|
|54.0|  F|          ATA|     54.0|    55.0|
|38.0|  M|          ASY|     38.0|    39.0|
|43.0|  F|          ATA|     43.0|    44.0|
|60.0|  M|          ASY|     60.0|    61.0|
|36.0|  M|          ATA|     36.

In [196]:
print(df.columns)

['age', 'sex', 'ChestPainType', 'RestingBP', 'AgeFixed']


In [197]:
# group by age
disease_by_age = df.groupby('age').mean().select(['age','avg(RestingBP)'])
# sort values in desnding order
from pyspark.sql.functions import desc
disease_by_age.orderBy(desc("age")).show(5)

+----+--------------+
| age|avg(RestingBP)|
+----+--------------+
|77.0|          77.0|
|76.0|          76.0|
|75.0|          75.0|
|74.0|          74.0|
|73.0|          73.0|
+----+--------------+
only showing top 5 rows



In [198]:
# aggregate function
from pyspark.sql import functions as F
df.agg(F.min(df['age']),F.max(df['age']),F.avg(df['RestingBP'])).show()

+--------+--------+------------------+
|min(age)|max(age)|    avg(RestingBP)|
+--------+--------+------------------+
|    28.0|    77.0|53.510893246187365|
+--------+--------+------------------+



In [199]:
# run an SQL query on the data
df.createOrReplaceTempView("df") # tell PySpark how the table will be called in the SQL query
spark.sql("""SELECT sex from df""").show(2)

# we also choose columns using SQL sytnx, with a command that combins '.select()' and '.sql()'
df.selectExpr("age >= 40 as older", "age").show(2)

+---+
|sex|
+---+
|  M|
|  F|
+---+
only showing top 2 rows

+-----+----+
|older| age|
+-----+----+
| true|40.0|
| true|49.0|
+-----+----+
only showing top 2 rows



In [201]:
df.groupby('age').pivot('sex', ("M", "F")).count().show(3)

+----+---+---+
| age|  M|  F|
+----+---+---+
|64.0| 16|  6|
|47.0| 15|  4|
|58.0| 35|  7|
+----+---+---+
only showing top 3 rows



In [202]:
# pivot - expensive operation
df.selectExpr("age >= 40 as older", "age",'sex').groupBy("sex")\
                    .pivot("older", ("true", "false")).count().show()

+---+----+-----+
|sex|true|false|
+---+----+-----+
|  F| 174|   19|
|  M| 664|   61|
+---+----+-----+

